In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv',index_col='Id')
test=pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv',index_col='Id')

In [ ]:
train.isna().sum()

In [ ]:
train.info()

In [ ]:
train.hist(figsize=(20,15))

In [ ]:
train.Cover_Type.value_counts()

In [ ]:
train=train[train['Cover_Type']!=5]

In [ ]:
train["Aspect"][train["Aspect"] < 0] += 360
train["Aspect"][train["Aspect"] > 359] -= 360


test["Aspect"][test["Aspect"] < 0] += 360
test["Aspect"][test["Aspect"] > 359] -= 360

In [ ]:
def aspect(x):
    if (x>=0 and x<=22.5) or (x>=337.5):
        return 'N'
    if x>=22.5 and x<=67.5:
        return 'NE' 
    if x>=67.5 and x<=112.5:
        return 'E' 
    if x>=67.5 and x<=157.5:
        return 'SE'
    if x>=157.5 and x<=202.5:
        return 'S' 
    if x>=202.5 and x<=247.5:
        return 'SW' 
    if x>=247.5 and x<=292.5:
        return 'W' 
    if x>=292.5 and x<=337.5:
        return 'NW' 
train['Aspect_categories']=train.Aspect.apply(aspect)
test['Aspect_categories']=test.Aspect.apply(aspect)

In [ ]:
train.Slope=train.Slope.abs()
test.Slope=test.Slope.abs()

In [ ]:
train.Horizontal_Distance_To_Hydrology=train.Horizontal_Distance_To_Hydrology.abs()
train.Vertical_Distance_To_Hydrology=train.Vertical_Distance_To_Hydrology.abs()
train.Horizontal_Distance_To_Roadways=train.Horizontal_Distance_To_Roadways.abs()
train.Horizontal_Distance_To_Fire_Points=train.Horizontal_Distance_To_Fire_Points.abs()
train.Hillshade_9am=train.Hillshade_9am.clip(0,255)
train.Hillshade_3pm=train.Hillshade_3pm.clip(0,255)
train.Hillshade_Noon=train.Hillshade_Noon.clip(0,255)

test.Horizontal_Distance_To_Hydrology=test.Horizontal_Distance_To_Hydrology.abs()
test.Vertical_Distance_To_Hydrology=test.Vertical_Distance_To_Hydrology.abs()
test.Horizontal_Distance_To_Roadways=test.Horizontal_Distance_To_Roadways.abs()
test.Horizontal_Distance_To_Fire_Points=test.Horizontal_Distance_To_Fire_Points.abs()
test.Hillshade_9am=test.Hillshade_9am.clip(0,255)
test.Hillshade_3pm=test.Hillshade_3pm.clip(0,255)
test.Hillshade_Noon=test.Hillshade_Noon.clip(0,255)

In [ ]:
train['Distance_To_Hydrology']=(train.Horizontal_Distance_To_Hydrology**2+train.Vertical_Distance_To_Hydrology)**2
test['Distance_To_Hydrology']=(test.Horizontal_Distance_To_Hydrology**2+test.Vertical_Distance_To_Hydrology)**2

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
for_clusters=sc.fit_transform(train[['Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology']])
test_clst=sc.transform(test[['Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology']])

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
distortions=[]
for i in range(1,10):
    km=KMeans(n_clusters=i)
    km.fit(for_clusters)
    distortions.append(km.inertia_)
plt.plot(distortions)

In [ ]:
km=KMeans(n_clusters=3)
km.fit(for_clusters)

In [ ]:
train['Hydrology_categories']=km.labels_
test['Hydrology_categories']=km.predict(test_clst)

In [ ]:
train[['Soil_Type'+str(i) for i in range(1,41)]].sum(axis=1)

In [ ]:
train['Count_of_Soils']=train[['Soil_Type'+str(i) for i in range(1,41)]].sum(axis=1)
test['Count_of_Soils']=test[['Soil_Type'+str(i) for i in range(1,41)]].sum(axis=1)

In [ ]:
train['Count_of_Wilderness_Areas']=train[['Wilderness_Area'+str(i) for i in range(1,5)]].sum(axis=1)
test['Count_of_Wilderness_Areas']=test[['Wilderness_Area'+str(i) for i in range(1,5)]].sum(axis=1)

In [ ]:
train=pd.get_dummies(train,columns=['Count_of_Soils','Hydrology_categories','Aspect_categories','Count_of_Wilderness_Areas']).drop('Aspect',axis=1)
test=pd.get_dummies(test,columns=['Count_of_Soils','Hydrology_categories','Aspect_categories','Count_of_Wilderness_Areas']).drop('Aspect',axis=1)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

In [ ]:
train= reduce_mem_usage(train)
test= reduce_mem_usage(test)

In [ ]:
test=test[set(test.columns)&set(train.columns)]
X_train=train[set(test.columns)&set(train.columns)]
y_train=train['Cover_Type']

In [ ]:
test=test.reindex(columns=X_train.columns)

In [ ]:
X_train=sc.fit_transform(X_train)
test=sc.transform(test)
y_train=pd.get_dummies(y_train).values
y_train

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import os
import warnings 
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #suppressing GPU warnings
os.environ["CUDA_VISIBLE_DEVICES"]='1'# GPU using on
def load_model(name:str):
    
    early_stopping = callbacks.EarlyStopping(
        patience=3,
        min_delta=0,
        monitor='val_loss',
        restore_best_weights=True,
        verbose=1,
        mode='min', 
        baseline=None,
    )

    plateau = callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=2, 
            verbose=0,
            mode='min')

# -----------------------------------------------------------------
# Model 

    model = keras.Sequential([
        layers.Dense(1024, activation = ACTIVATION, input_shape = [X_train.shape[1]]),
        layers.Dropout(.2),
        layers.BatchNormalization(),
        layers.Dense(512, activation =ACTIVATION), 
        layers.Dropout(.2),
        layers.BatchNormalization(),
        layers.Dense(64, activation =ACTIVATION),
        layers.Dense(6, activation='softmax'),
    ])
# -----------------------------------------------------------------

    model.compile(
        optimizer= keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['AUC'],
    )
    
    return model, early_stopping, plateau

In [ ]:
from sklearn.model_selection import KFold
preds_test = []
EPOCHS = 5
BATCH_SIZE = 128 
ACTIVATION = 'relu'
LEARNING_RATE = 0.000265713
FOLDS = 5
kf = KFold(n_splits=FOLDS,random_state=0,shuffle=True)
for train_index, valid_index in kf.split(X_train):
    X_train1,X_valid1 = X_train[train_index], X_train[valid_index]
    y_train1,y_valid1 = y_train[train_index], y_train[valid_index]
    model, early_stopping, plateau  = load_model('version1')
    history = model.fit(  X_train1, y_train1,
                validation_data = (X_valid1, y_valid1),
                batch_size = BATCH_SIZE, 
                epochs = EPOCHS,
                callbacks = [early_stopping, plateau],
                shuffle = True,
                verbose = 1,
              )
    preds_test.append(model.predict(test))

In [ ]:
predicts=np.array(preds_test)
preds_for_subm=np.argmax(predicts.mean(axis=0),axis=1)+1
preds_for_subm[(preds_for_subm==5)|(preds_for_subm==6)]+=1
subm=pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv')
subm['Cover_Type']=preds_for_subm
subm.to_csv('sub_featured.csv',index=None)